# Brest Cancer Wisconsin Dataset Example

This Jupyter Notebook demonstrates how to build and use a neural network with dense layers to process and classify data.

In [ ]:
import numpy as np
from ucimlrepo import fetch_ucirepo

from src import NeuralNetwork

from src.preprocessing import min_max_scaler, train_test_split
from src.core import Tensor
from src.activation import LeakyRelu, Sigmoid
from src.loss import BinaryCrossentropy
from src.optimizer import Adam
from src.scheduler import CosineScheduler
from src.config import FeedForwardConfig, TrainingConfig
from src.structure import Dense, Dropout

In [ ]:
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

X = Tensor(X.to_numpy())
y = Tensor(y.to_numpy(dtype=str))

In [ ]:
classes = tuple(np.unique(y))

# Normalize data
data = min_max_scaler(X, 0, 1)

# Combine data and labels into a structured array
combined = Tensor([(d, label[0]) for d, label in zip(data, y)], dtype=object)

# Split data into training and testing sets
train, test = train_test_split(combined, random_state=0)

In [ ]:
config = FeedForwardConfig(
    network_structure=[
        Dense(len(X[0])),
        Dropout(p=0.1),
        Dense(32),
        Dropout(p=0.1),
        Dense(16),
        Dropout(p=0.1),
        Dense(8),
        Dense(1)
    ],
    classes=classes,
    hidden_activation=LeakyRelu(),
    output_activation=Sigmoid(),
)

nn = NeuralNetwork(config)

nn.train(
    list(train),
    list(test),
    config=TrainingConfig(
        lr=CosineScheduler(learning_rate=0.001, min_lr=0.0001, max_steps=20, cyclic=True),
        patience_stop=100,
        loss=BinaryCrossentropy(),
        optimizer=Adam(),
        batch_size=4,
        debug=True,
        store=None,
    ),
)

print(f"final test: {nn.evaluate(list(combined))}, test: {nn.evaluate(list(test))}, train: {nn.evaluate(list(train))}")